# 距離学習のファインチューニング
## ライブラリのインストール

In [ ]:
%pip install pandas plotly torch torchvision scikit-learn plotly tqdm kaleido nbformat pytorch_metric_learning

In [ ]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
import plotly.express as px
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from pytorch_metric_learning import distances, losses, regularizers
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import datasets, transforms


## 学習コードの作成

In [ ]:
def get_random_sampler(dataset, subset_size=1000, random_seed=42):
    """データセットからランダムにデータを取得するためのSubsetRandomSamplerを作成する

    Args:
        dataset (_type_): 対象データセット
        subset_size (int, optional): ランダムに抽出するデータ数. Defaults to 1000.
        random_seed (int, optional): seed値. Defaults to 42.

    Returns:
        _type_: _description_
    """
    # データセットのインデックス配列を作成する
    indices = list(range(len(dataset)))
    # インデックスをシャッフルする
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    # シャッフルしたインデックスからsubset_size分だけ取得する
    subset_indices = indices[:subset_size]
    # SubsetRandomSamplerにインデックスを渡すことで、そのインデックスのデータをサンプリングする
    return SubsetRandomSampler(subset_indices)


def train_epoch(model, train_loader, criterion, optimizer, device):
    """1エポック分の学習を行う

    Args:
        model (_type_):
        train_loader (_type_):
        criterion (_type_):
        optimizer (_type_):
        device (_type_):

    Returns:
        _type_: _description_
    """
    # モデルをtrainモードにする
    model.train()
    # 損失を記録する変数を定義
    running_loss = 0.0

    # ミニバッチごとにループを回す
    for images, labels in tqdm(train_loader, total=len(train_loader)):
        images, labels = images.to(device), labels.to(device)

        # 勾配を初期化する
        optimizer.zero_grad()
        # 準伝搬
        outputs = model(images)
        # 損失関数を計算
        loss = criterion(outputs, labels)
        # 逆伝搬
        loss.backward()
        # パラメータ更新
        optimizer.step()

        # ミニバッチの損失を計算し記録する
        running_loss += loss.item()

    # 1エポックあたりの平均損失を計算する
    avg_loss = running_loss / len(train_loader)
    return avg_loss


def validate_epoch(model, val_loader, criterion, device):
    """1エポック分の検証を行う

    Args:
        model (_type_): _description_
        val_loader (_type_): _description_
        criterion (_type_): _description_
        device (_type_): _description_

    Returns:
        _type_: _description_
    """
    # モデルをevalモードにする
    model.eval()
    # 損失を記録する変数を定義
    running_loss = 0.0

    all_output = []
    all_labels = []
    # 勾配計算をしないようにする(推論なので)
    with torch.no_grad():
        # ミニバッチごとにループを回す
        for images, labels in tqdm(val_loader, total=len(val_loader)):
            # デバイスの指定
            images, labels = images.to(device), labels.to(device)
            # 準伝搬
            outputs = model(images)
            all_output.append(outputs)
            all_labels.append(labels)
            # 損失計算
            loss = criterion(outputs, labels)
            # 損失を記録する
            running_loss += loss.item()

    # 1エポックあたりの平均損失を計算する
    avg_loss = running_loss / len(val_loader)
    # テストデータの予測結果を取得する
    all_output = torch.cat(all_output, dim=0).cpu()
    all_labels = torch.cat(all_labels, dim=0).cpu()
    return avg_loss, all_output, all_labels


def get_cifar10_train_test_loader(
    train_samples: int = 1000,
    test_samples: int = 1000,
    resize: tuple[int, int] = (256, 256),
    batch_size: int = 32,
):
    """CIFAR-10データセットの学習データと検証データのDataLoaderを作成する

    Args:
        train_samples (int, optional): _description_. Defaults to 1000.
        test_samples (int, optional): _description_. Defaults to 1000.
        resize (tuple[int, int], optional): _description_. Defaults to (256, 256).
        batch_size (int, optional): _description_. Defaults to 32.

    Returns:
        _type_: _description_
    """
    # 画像を256x156にリサイズして、テンソルに変換する
    transform = transforms.Compose([transforms.Resize(resize), transforms.ToTensor()])

    # 学習データセットの作成
    train_dataset = datasets.CIFAR10(
        root="./data", train=True, download=True, transform=transform
    )
    # データセットからランダムにデータを取得する
    train_sampler = get_random_sampler(train_dataset, train_samples)
    # 学習DataLoaderの作成
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler
    )

    # 検証データセットの作成
    test_dataset = datasets.CIFAR10(
        root="./data", train=False, download=True, transform=transform
    )
    # データセットからランダムにデータを取得する
    test_sampler = get_random_sampler(test_dataset, test_samples)
    # 検証DataLoaderの作成
    test_loader = DataLoader(test_dataset, batch_size=batch_size, sampler=test_sampler)
    return train_loader, test_loader


## データのダウンロード


In [ ]:
# データをロードする
train_loader, test_loader = get_cifar10_train_test_loader(
    train_samples=5000, test_samples=1000
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## モデルの作成

In [ ]:
def get_model(pretrained: bool = True, state_dict: dict | None = None):
    # 距離学習
    # 事前学習済みのResNetモデルをロード
    model = models.resnet50(pretrained=pretrained)
    # ResNetの最後の全結合層をembedding数に置き換え
    model.fc = nn.Linear(model.fc.in_features, 128)
    if state_dict is not None:
        model.load_state_dict(state_dict)
    # デバイスの設定
    model.to(device)
    # ArcFace lossの設定
    # コサイン類似度を使う
    distance = distances.CosineSimilarity()
    regularizer = regularizers.RegularFaceRegularizer()
    criterion = losses.ArcFaceLoss(
        num_classes=10,
        embedding_size=128,
        margin=28.6,
        scale=64,
        weight_regularizer=regularizer,
        distance=distance,
    )
    # GPUが使えるなら使う
    if device != "cpu":
        criterion = criterion.cuda()

    # オプティマイザの設定
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    return model, criterion, optimizer


## モデルの学習

In [ ]:
# モデルの出力をKNNでラベルに変換する
def eval(model, train_loader, test_loader):
    # モデルをevalモードにする
    model.eval()

    x_train = []
    x_test = []
    y_train = []
    y_test = []
    with torch.no_grad():
        # 学習データの推論結果を得る
        for x_org, y in tqdm(train_loader, total=len(train_loader)):
            # デバイスの指定
            x_org, y = x_org.to(device), y.to(device)
            # モデルでx_orgを新しい空間に写像
            x = model(x_org)
            x_train.append(x)
            y_train.append(y)
        # テストデータの推論結果を得る
        for x_org, y in tqdm(test_loader, total=len(test_loader)):
            # デバイスの指定
            x_org, y = x_org.to(device), y.to(device)
            x = model(x_org)
            x_test.append(x)
            y_test.append(y)
    # データを変換する
    x_train = torch.cat(x_train).cpu().numpy()
    x_test = torch.cat(x_test).cpu().numpy()
    y_train = torch.cat(y_train).cpu().numpy()
    y_test = torch.cat(y_test).cpu().numpy()

    # KNNモデルを作成
    knn = KNeighborsClassifier(n_neighbors=5, metric="cosine")
    # KNNモデルを学習データの結果で学習する
    knn.fit(x_train, y_train)

    # テストデータの推定ラベルをKNNモデルで推論
    y_pred = knn.predict(x_test)

    return {
        "x_train": x_train,
        "x_test": x_test,
        "y_train": y_train,
        "y_test": y_test,
        "y_pred": y_pred,
    }


def run(
    pretrained: bool = True,
    num_epochs: int = 100,
):
    # 距離学習
    # 事前学習済みのResNetモデルをロード
    model, criterion, optimizer = get_model(pretrained=pretrained)

    result = []
    # 出力先のディレクトリを作成
    output_dir = Path(
        "output", "metric_learning", "pretrained" if pretrained else "un_pretrained"
    )
    os.makedirs(output_dir, exist_ok=True)
    # 学習
    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, output, labels = validate_epoch(model, test_loader, criterion, device)

        print(
            f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}"
        )
        # KNNで評価する
        knn_result = eval(model, train_loader, test_loader)
        # ラベルの正解率
        label_acc = (knn_result["y_pred"] == knn_result["y_test"]).sum() / len(
            knn_result["y_test"]
        )
        result.append(
            {"train_loss": train_loss, "val_loss": val_loss, "val_acc": label_acc}
        )
        # 結果を保存する
        torch.save(
            {"output": output, "label": labels, "pred_labels": knn_result["y_pred"]},
            output_dir / f"epoch_{epoch}_output.pt",
        )

    df_result = pd.DataFrame(result)
    df_result.to_csv(output_dir / "training_curve.csv")



In [ ]:
run(pretrained=True, num_epochs=10)
# run(train_samples=5000, test_samples=1000, pretrained=False)

## モデルの評価
学習済みモデルをロードし、テストデータで評価する。

In [ ]:
def data_loader_to_array(data_loader):
    images = []
    labels = []
    for image, label in data_loader:
        images.append(image)
        labels.append(label)

    data = torch.cat(images)
    label = torch.cat(labels)

    data_reshaped = data.view(data.shape[0], -1).numpy()
    return data_reshaped, label


In [ ]:
## t-SNEによる次元圧縮
data_reshaped, label = data_loader_to_array(test_loader)


def plot_tsne(data, label):
    tsne = TSNE(n_components=2, random_state=0)
    data_tsne = tsne.fit_transform(data)
    df = pd.DataFrame(data_tsne, columns=["x", "y"])
    df["label"] = label.numpy().astype(str)
    df = df.sort_values("label")

    fig = px.scatter(
        df,
        x="x",
        y="y",
        title="t-SNE Visualization of Image Data",
        color="label",
    )
    fig.update_layout(legend_title="label")
    return fig



In [ ]:
fig = plot_tsne(data_reshaped, label)
fig.write_image("output/metric_learning/tsne_org.png")
fig

In [ ]:

outputs = sorted(list(Path("output/metric_learning/pretrained").glob("**/epoch_*")))
print(outputs)

for i, output in tqdm(enumerate(outputs)):
    if i % 10 != 0:
        continue
    loaded_data = torch.load(output)
    output_data = loaded_data["output"]
    true_label = loaded_data["label"]

    fig = plot_tsne(output_data, true_label)
    fig.write_image(f"output/metric_learning/tsne_epoch_{i}.png")
fig


In [ ]:
px.line(
    pd.read_csv("output/metric_learning/pretrained/training_curve.csv"),
    y=["train_loss", "val_loss"],
)

In [ ]:
fig = px.line(
    pd.read_csv("output/metric_learning/pretrained/training_curve.csv"),
    y=["val_acc"],
)
fig.update_layout(title="KNNによるAccuracy")
fig.update_yaxes(title="validation Accuracy")
fig.update_xaxes(title="epoch")
fig.write_image("output/metric_learning/pretrained/accuracy.png")